# Deploy your Python Model as a RESTful API
This tutorial demonstrates how to deploy an arbitrary python function as an api with IBM Cloud and Flask -- complete with clean, intuitive Swagger API documentation. Our python function will be a simple implementation of the <a href="https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes" target="_blank" rel="noopener no referrer">Sieve of Eratosthenes</a>, which takes one integer parameter $n$ and returns all primes $p$ such that $p \leq{n}$. 

While completing this tutorial, you will:

 - Create an algorithm (or some arbitrary function) using Notebooks in Watson Studio
 - Persist the function
 - Develop a RESTful API with Swagger documentation using Flask
 - Deploy your API to IBM Cloud
 
Be sure to read the [companion article on our blog](https://datascience.ibm.com/blog/deploy-your-python-functions-as-a-rest-api/). 

This notebook runs on Python with Spark.

## Table of contents
1. [Implement the "Sieve of Eratosthenes"](#implement)<br/>
2. [Develop a RESTful API](#restfulapi)<br/> 
3. [Include the supporting documents](#supportingdocs)  
4. [Deploy to IBM Cloud](#deploy)
5. [Test the API](#test)

## 1. Implement the "Sieve of Eratosthenes"<a class="anchor" id="implement"></a>
First, we need to create a folder which will contain all of the files needed for our application. I'll name mine `sieve`. 

In [1]:
!mkdir sieve

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20180904082423-0000


Change to that directory and let's get started. 

In [2]:
cd sieve

/home/spark/shared/sieve


The Sieve of Eratosthenes is an ancient prime number sieve which, roughly, finds all the primes up to a given upper bound by striking multiples from the list of candidate primes. The algorithm looks like this:

![alt_text](https://upload.wikimedia.org/wikipedia/commons/b/b9/Sieve_of_Eratosthenes_animation.gif) [[1]](https://en.wikipedia.org/wiki/GNU_Free_Documentation_License)

Let's write the function in pure Python. The exact implementation is not the focus of this article, but it is cool to think about. 

In [3]:
def Eratosthenes(upper_bound):
    # Return a list of the primes below the upper_bound
    # first, create a list with all primes set to `True`
    prime = [True] * upper_bound
    for p in range(3, upper_bound, 2):
        if p > (upper_bound**.5):
            break
        if prime[p]==True:
            # remember that all multiples of p are not prime, whether or not p is prime
            for i in range(p * p, upper_bound, 2 * p):
                prime[i] = False
    # don't forget about the number "2"
    return [2] + [p for p in range(3, upper_bound, 2) if prime[p]]

In [4]:
Eratosthenes(22)

[2, 3, 5, 7, 11, 13, 17, 19]

We can use some <a href="http://ipython.readthedocs.io/en/stable/interactive/magics.html" target="_blank" rel="noopener no referrer">built-in Jupyter magic</a> to write files. We'll put our function in this file for use later in the application. You can define as many functions as you'd like in this file. 

To write, simply prepend this line to the code cell that we want to write `%%write_file <file_name>` 

We'll name this `prime_sieve.py`. 

In [5]:
%%writefile prime_sieve.py
def Eratosthenes(upper_bound):
    # Return a list of the primes below the upper_bound
    # first, create a list with all values set to true
    prime = [True] * upper_bound
    for p in range(3, upper_bound, 2):
        if p > (upper_bound**.5):
            break
        if prime[p]:
            # remember that all multiples of p are not prime, whether or not p is prime
            for i in range(p * p, upper_bound, 2 * p):
                prime[i] = False
    # don't forget about the number "2"
    return [2] + [p for p in range(3, upper_bound, 2) if prime[p]]

Writing prime_sieve.py


We'll save this function into a file, so we can make use of it later.

## 2. Develop a RESTful API<a class="anchor" id="restfulapi"></a>
We'll use a few tools to develop the API, like <a href="https://www.cloudfoundry.org" target="_blank" rel="noopener no referrer">Cloud Foundry</a> and <a href="http://flask.pocoo.org/" target="_blank" rel="noopener no referrer">Flask</a>. This part of the tutorial is an expansion of a <a href="https://github.com/IBM-Bluemix/get-started-python" target="_blank" rel="noopener no referrer">previous guide</a>. From within a Notebook, we can write files and execute shell commands, which means that we can develop the app entirely within Watson Studio notebooks. 


In [6]:
%%writefile my_flask_app.py
from flask import Flask, Response, jsonify
from flask_restplus import Api, Resource, fields, reqparse
from flask_cors import CORS, cross_origin
import os

# the app
app = Flask(__name__)
CORS(app)
api = Api(app, version='1.0', title='APIs for Python Functions', validate=False)
# we can create namespace to organize the api and docs
ns = api.namespace('primality', 'Returns a list of all primes below a given upper bound')

# load the algo
from prime_sieve import Eratosthenes as algo

"""

We import our function `Erasosthenes` from the file prime_sieve.py. 
You create all the classes and functions that you want in that file, 
and import them into the app. 

"""

# model the input data
model_input = api.model('Enter the upper bound:', {
    "UPPER_BOUND": fields.Integer(maximum=10e16)})

# the input data type here is Integer. You can change this to whatever works for your app.

# On IBM Cloud, get the port number from the environment variable PORT
# When running this app on the local machine, default the port to 8080

port = int(os.getenv('PORT', 8080))


# The ENDPOINT
@ns.route('/sieve') # the endpoint
class SIEVE(Resource):
    @api.response(200, "Success", model_input) # return a formatted response
    @api.expect(model_input) # expcect the required the input data
    def post(self): # prefer POST
        parser = reqparse.RequestParser() # parse the args
        parser.add_argument('UPPER_BOUND', type=int) # get the data
        args = parser.parse_args()
        inp = int(args["UPPER_BOUND"]) # our input data
        result = algo(inp) # apply algo
        return jsonify({"primes": result})
# run      
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=port, debug=True)

Writing my_flask_app.py


## 3. Include the supporting documents<a class="anchor" id="supportingdocs"></a>
We'll need to include some other files in order for our api to behave properly when we try to deploy the app to IBM Cloud. These files are:
- `manifest.yml`, this includes basic information about your app, such as the name and memory 
- `setup.py`
- `README.md`
- `requirements.txt`
- `Procfile`


In [7]:
%%writefile manifest.yml
---
applications:
 - name: PRIMALITY_UNIQUE # name your app whatever you'd like
   random-route: true
   memory: 256M # can adjust this based on your service 

Writing manifest.yml


In [8]:
%%writefile Procfile
web: python my_flask_app.py

Writing Procfile


In [9]:
%%writefile README.md
"Getting Started with Python Algos on IBM Cloud"

Writing README.md


In [10]:
%%writefile requirements.txt
Flask==0.11.1
cloudant==2.4.0
flasgger==0.6.4
Flask-Cors==3.0.2
Flask-RESTful==0.3.6
flask-restplus==0.9.2
gevent==1.2.1

Writing requirements.txt


In [11]:
%%writefile setup.py
"""
Hello World app for deploying Python functions as APIs on IBM Cloud
"""

# Always prefer setuptools over distutils
from setuptools import setup, find_packages
# To use a consistent encoding
from codecs import open
from os import path

here = path.abspath(path.dirname(__file__))

# Get the long description from the README file
with open(path.join(here, 'README.md'), encoding='utf-8') as f:
    long_description = f.read()

setup(
    name='primality_test',
    version='1.0.0',
    description='Running Python apps on IBM Cloud',
    long_description=long_description,
    url='https://github.com/IBM-Cloud/get-started-python',
    license='Apache-2.0'
)


Writing setup.py


In [12]:
%%writefile LICENSE
"""Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION ..."""

Writing LICENSE


## 4. Deploy to IBM Cloud<a class="anchor" id="deploy"></a>
Now, we'll deploy the application to IBM Cloud. To accomplish this, you'll need a <a href="https://console.bluemix.net/registration/" target="_blank" rel="noopener no referrer">IBM Cloud Account</a>. For more information on this part of the tutorial, we have some <a href="https://www.ibm.com/blogs/bluemix/2015/03/simple-hello-world-python-app-using-flask/" target="_blank" rel="noopener no referrer">helpful documentation</a> on deploying an app after it's been created. 

We have written all of the files needed for our app to run. In order to deploy to IBM Cloud (and retain the most felixibility when it comes to debugging, etc), I suggest that you copy this notebook and run all the previous cells. If you naviagte to `Kernel` and select `Restart & Run all`, this will happen automatically. Then, in your local directory, you'll have all the necessary files to proceed to these next steps. 

One alternative is to create the above files yourself with your favorite text editor. 

Ok, let's deploy this app. You'll need basic familiarity with the command line for best results. 

1. Install <a href="https://github.com/cloudfoundry/cli/releases?cm_mc_uid=85234897373114979704601&cm_mc_sid_50200000=1497971073&cm_mc_sid_52640000=1497900876" target="_blank" rel="noopener no referrer">Cloud Foundry command line</a>. This open source service works with IBM Cloud to deploy apps. 

2. Open the terminal or command prompt. 
    - Mac – Click the finder icon in the top right, search for terminal and open that
    - Linux – Depends on the distro, but you probably know where it is. In the menu look
    for accessories. 
    - Windows – Click the start button, search for cmd, open. 
    
3. Log in to IBM Cloud and run the following command: 

   `cf login -a https://api.ng.bluemix.net`
   
   ![login1](https://datascience.ibm.com/blog/content/images/2017/06/Screen-Shot-2017-06-20-at-4.27.35-PM.png)
   
4. Enter your username and password. 

5. Push the app!
   
    `cf push <my_app_name>`
   
6. Replace your `<my_app_name>` with the name of the app in the `manifest.yml` file. Make sure your app name is unique. 

   ![log2](https://datascience.ibm.com/blog/content/images/2017/06/Screen-Shot-2017-06-20-at-4.38.21-PM.png)
   

That's it! The client returns a URL where you can find the API and documentation. Alternatively, you can log in to IBM Cloud with a Web browser and manage your API there. 

## 5. Test the API<a class="anchor" id="test"></a>

In [13]:
import requests

.... and read the docs!

![the_docs](https://datascience.ibm.com/blog/content/images/2017/06/Screen-Shot-2017-06-20-at-4.47.14-PM.png)

On a more serious note, you've created the documentation yourself. Just navigate to the IBM Cloud URL. You can interact with the documentation to test how different requests might be handled. Moving forward, the method we followed in this tutorial will work well for more complex python applications. Watson Studio makes it easy to prototype and deploy new models and applications. 

Stay tuned for more tutorials on the newest features of Watson Studio! 

### Authors

**Adam Massachi** is a Data Science Intern with the Watson Studio and IBM Watson team at IBM. Before IBM, he worked on political campaigns for Democrats, building and managing large volunteer operations and contributing to campaign finance initiatives. Say hello [@adammassach](https://twitter.com/AdamMassach?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor)!

### Citations

[[1]](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes) The Sieve of Eratosthenes Wikipedia entry contains the graphic used above, available under the terms of the CC-BYSA 3.0 License. 

Copyright &copy; IBM Corp. 2017, 2018. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>